In [4]:
from gff_analyser import gffClasses, gffBuilder
import hdf5plugin
from pybedtools import BedTool
import os
import sys
import pandas as pd
import episcanpy.api as epi
from sctoolbox import annotation
import sctoolbox.atac as atac
import sctoolbox.calc_overlap_pct as overlap
from sctoolbox import celltype_annotation

In [2]:
GTF_PATH = '/home/stud3/notebooks/homo_sapiens.104.mainChr.gtf'
gtf_gene= '/home/stud3/notebooks/out/homo_sapiens.104.mainChr.gene.gtf'
gtf_exon = '/home/stud3/notebooks/out/homo_sapiens.104.mainChr.exon.gtf'
five = '/home/stud3/notebooks/out/homo_sapiens.104.mainChr.five_prime_utr.gtf'
three = '/home/stud3/notebooks/out/homo_sapiens.104.mainChr.three_prime_utr.gtf'
frag = '/home/stud3/notebooks/stomach_SM-JF1O3_rep1_fragments_sorted.bed'
FRAG_FILE_sorted = '/home/stud3/notebooks/stomach_SM-JF1O3_rep1_fragments_sorted.bed'
INPUT_PATH = '/mnt/workspace_stud/catlas_ref/cellXcCRE/'
H5AD = 'stomach_SM-JF1O3.h5ad' 
test = '/home/stud3/notebooks/stomach_more_cluster.h5ad'

In [ ]:
object_list = gffBuilder.build_gff3_class(file=GTF_PATH)

for element in object_list:
    features = element.count_features()

    element.generate_feature_gtf(gffdata_list=object_list, feature_keys=features)
    element.generate_promotor_gtf(gffdata_list=object_list, promotor_distance=2000)
    element.generate_tss_gtf(gffdata_list=object_list, tss_distance=100)

In [ ]:
adata = epi.read_h5ad(INPUT_PATH + H5AD)
adata

In [5]:
adata = epi.read_h5ad(test)

In [7]:
adata.obs

,logUMI,tsse,tissue,cell type,Life stage,closest Cell Ontology term(s),Cell Ontology ID,insertsize_count,mean_insertsize,n_total_fragments,...,n_fragments_in_enhancer,pct_fragments_in_enhancer,n_fragments_in_five_prime_utr,pct_fragments_in_five_prime_utr,n_fragments_in_TSS,pct_fragments_in_TSS,nb_features,log_nb_features,leiden,SCSA_pred_celltype
index,,,,,,,,,,,,,,,,,,,,,
GGAACACTAGTTAGGGACCCTC,3.736635,9.715475,stomach_SM-JF1O3,Small Intestinal Enterocyte,Adult,enterocyte of epithelium of small intestine,CL:1000334,16274,178.19,16274,...,6.0,0.000369,696,0.042768,680,0.041784,549.0,3.252125,6,Meiotic prophase fetal germ cell
GGGACTATTCCAAATCGAAAGC,3.051153,7.523511,stomach_SM-JF1O3,Small Intestinal Enterocyte,Adult,enterocyte of epithelium of small intestine,CL:1000334,3376,154.95,3376,...,2.0,0.000592,67,0.019846,76,0.022512,60.0,2.513218,8,NaN
AAACGCAAGCCGCCGAATTACT,3.790215,8.695652,stomach_SM-JF1O3,Naive T cell,Adult,naive t cell,CL:0000898,18555,135.09,18555,...,9.0,0.000485,1043,0.056211,679,0.036594,932.0,3.378943,4,Mesenchymal cell
AAACGCAAGCGTCCGGTCCAAA,3.105169,19.151515,stomach_SM-JF1O3,Naive T cell,Adult,naive t cell,CL:0000898,4315,151.45,4315,...,0.0,0.000000,760,0.176130,575,0.133256,711.0,2.993877,1,T cell
AAACGCAAGCTCGGTACTAACT,3.029384,18.181818,stomach_SM-JF1O3,Naive T cell,Adult,naive t cell,CL:0000898,4262,157.27,4262,...,8.0,0.001877,631,0.148053,637,0.149460,393.0,2.814913,1,T cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CAGGCCTGTAACCCGATGGCTA,3.245019,8.522727,stomach_SM-JF1O3,Smooth Muscle (Colon) 1,Adult,enteric smooth muscle cell,CL:0002504,5115,141.34,5115,...,4.0,0.000782,293,0.057283,226,0.044184,279.0,2.871573,7,Macrophage
CCACTCCGAATGCAGATCGGCT,3.179264,12.561983,stomach_SM-JF1O3,Smooth Muscle (Colon) 1,Adult,enteric smooth muscle cell,CL:0002504,5074,128.69,5074,...,0.0,0.000000,241,0.047497,516,0.101695,275.0,2.811575,8,NaN
GAAATCCGCCGACTTACTGCCG,3.132580,18.475073,stomach_SM-JF1O3,Smooth Muscle (Colon) 1,Adult,enteric smooth muscle cell,CL:0002504,4368,147.05,4368,...,4.0,0.000916,628,0.143773,539,0.123397,585.0,3.011147,4,Mesenchymal cell


In [ ]:
#overlap.pct_fragments_in_promotors(adata, gtf_file=GTF_PATH, fragments_file=FRAG_FILE_sorted, bam_file=None, cb_col=None, species=None, feature='peak20')

In [ ]:
# Getting the gene body from gene
bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')
intersect_cmd = f'{bedtools} subtract -a {gtf_gene} -b {five} {three}  > out/homo_sapiens.104.mainChr.gene.gene_bodies.gtf'
os.system(intersect_cmd)

In [ ]:
# Getting the gene body from exon
bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')
intersect_cmd = f'{bedtools} subtract -a {gtf_exon} -b {five} {three}  > out/homo_sapiens.104.mainChr.exon_gene_bodies.gtf'
os.system(intersect_cmd)

In [ ]:
# Getting the enhancer from gtf reference
bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')
intersect_cmd = f'{bedtools} intersect -a {gtf_gene} -b stomach_SM.enhancer.bed -wa -wb > out/homo_sapiens.104.mainChr.enhancer.gtf'
os.system(intersect_cmd)

In [ ]:
bed_df = pd.read_table(frag, header=None, names=["chrom", "start", "end", "feature", "count", "strand"])

# Filter the dataframe based on a count threshold
threshold = 10
filtered_df = bed_df[bed_df["count"] > threshold]


In [ ]:
def filter_peaks_from_bed(frag_file=frag ,thres=10):

    # Load the bed file into a pandas dataframe
    bed_df = pd.read_table(frag_file, header=None, names=["chrom", "start", "end", "feature", "count", "strand"])

    # Filter the dataframe based on a count threshold
    threshold = thres
    filtered_df = bed_df[bed_df["count"] > threshold]
    
    filename = f"{frag}.peak{thres}.bed"
    # Print the filtered dataframe
    filtered_df.to_csv(filename ,sep='\t', index=None)
    return thres


In [ ]:
tresh = filter_peaks_from_bed()

In [ ]:
peakfile = f"stomach_SM-JF1O3_rep1_fragments_sorted.bed.peak{tresh}.bed"

In [ ]:
# Getting the gene body from exon
bedtools = os.path.join('/'.join(sys.executable.split('/')[:-1]),'bedtools')
intersect_cmd = f'{bedtools} intersect -a {GTF_PATH} -b {peakfile} -wa -wb > out/homo_sapiens.104.mainChr.peak{tresh}.gtf'
os.system(intersect_cmd)